<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 8 [Assessment]:** RAG Evaluation</font>

<br>

Welcome to the last notebook of the course! In the previous notebook, you integrated a vector store solution into a RAG pipeline! In this notebook, you will take that same pipeline and evaluate it using numerical RAG evaluation techniques incorporating LLM-as-a-Judge metrics!

<br>

### **Learning Objectives:**

- Learn how to integrate the techniques from prior notebooks to numerically approximate the goodness of your RAG pipeline.

- **Final Exercice**: ***By working through this notebook in the Course Environment,* you will be able to submit the coding component of the course!**

<br>

### **Questions To Think About:**

- As you go along, remember what our metrics actually represent. Should our pipeline pass these objectives? Is our judge LLM sufficient for evaluating the pipeline? Does a particular metric even matter for our use case?
- If we left the vectorstore-as-a-memory component in our chain, do you think it would still pass the evaluation? Additionally, is the evaluation useful for assessing vectorstore-as-a-memory performance? 

<br>

### **Notebook Source:**

- This notebook is part of a larger [**NVIDIA Deep Learning Institute**](https://www.nvidia.com/en-us/training/) course titled [**Building RAG Agents with LLMs**](https://learn.next.courses.nvidia.com/courses/course-v1:DLI+S-FX-15+V1/about). If sharing this material, please give credit and link back to the original course.

<br>

### **Environment Setup:**

In [ ]:
%%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu ragas

## If you're in colab and encounter a typing-extensions issue,
##  restart your runtime and try again
# from langchain_nvidia_ai_endpoints._common import NVEModel

In [1]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try: 
        assert not hard_reset
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models

Retrieved NVIDIA_API_KEY beginning with "nvapi-QpK..."


{'playground_llama2_code_34b': 'df2bee43-fb69-42b9-9ee5-f4eabbeaf3a8',
 'playground_llama2_13b': 'e0bb7fb9-5333-4a27-8534-c6288f921d3f',
 'playground_sdxl': '89848fb8-549f-41bb-88cb-95d6597044a4',
 'playground_mixtral_8x7b': '8f4118ba-60a8-4e6b-8574-e38a4067a4a3',
 'playground_fuyu_8b': '9f757064-657f-4c85-abd7-37a7a9b6ee11',
 'playground_yi_34b': '347fa3f3-d675-432c-b844-669ef8ee53df',
 'playground_nemotron_steerlm_8b': '1423ff2f-d1c7-4061-82a7-9e8c67afd43a',
 'playground_neva_22b': '8bf70738-59b9-4e5f-bc87-7ab4203be7a0',
 'playground_deplot': '3bc390c7-eeec-40f7-a64d-0c6a719985f7',
 'playground_llama_guard': 'b34280ac-24e4-4081-bfaa-501e9ee16b6f',
 'playground_sdxl_turbo': '0ba5e4c7-4540-4a02-b43a-43980067f4af',
 'playground_clip': '8c21289c-0b18-446d-8838-011b7249c513',
 'playground_cuopt': '8f2fbd00-2633-41ce-ab4e-e5736d74bff7',
 'playground_nemotron_qa_8b': '0c60f14d-46cb-465e-b994-227e1c3d5047',
 'playground_llama2_code_13b': 'f6a96af4-8bf9-4294-96d6-d71aa787612e',
 'playground_n

----

<br>

## **Part 1:** Pre-Release Evaluation

In our previous notebook, we successfully combined several concepts to create a document chatbot with the aim of responsive and informative interactions. However, the diversity of user interactions necessitates comprehensive testing to truly understand the chatbot's performance. Thorough testing in varied scenarios is crucial to ensure that the system is not only robust and versatile but also aligns with user and provider expectations.

After defining your chatbot's roles and implementing the necessary features, evaluating it becomes a multi-stage process:

- **Typical Use Inspection:** Start by testing scenarios most relevant to your use case. See if your chatbot can reliably navigate discussions with limited human intervention.

    - Additionally, identify limitations or compartments that should be redirected to a human for inspection/supervision (i.e., human swap-in to confirm transactions or perform sensitive navigation) and implement those options.

- **Edge Case Inspection:** Explore the boundaries of typical use, identifying how the chatbot handles less common but plausible scenarios.

    - Before any public release, assess critical boundary conditions that could pose liability risks, such as the potential generation of inappropriate content.

    - Implement well-tested guardrails on all outputs (and possibly inputs) to limit undesired interactions and redirect users into predictable conversation flows.

- **Progressive Rollout:** Rolling out your model to a limited audience (first internal, then [A/B](https://en.wikipedia.org/wiki/A/B_testing)) and implement analytics features like usage analytics dashboards and feedback avenues (flag/like/dislike/etc).

Of these three steps, the first two can be done by a small team or an individual and should be iterated on as part of the development process. Unfortunately, this needs to be done frequently and can be prone to human error. **Luckily for us, LLMs can be used to help out with LLM-as-a-Judge formulations!**

*(Yeah, this probably isn't surprising by now. LLMs being strong is why this course is here...).*

----

<br>

## **Part 2:** LLM-as-a-Judge Formulation

In the realm of conversational AI, using LLMs as evaluators or 'judges' has emerged as a useful approach for configurable automatic testing of natural language task performance:

- An LLM can simulate a range of interaction scenarios and generate synthetic data, allowing an evaluation developer to generate targeted inputs to eliciting a range of behaviors from your chatbot.

- The chatbot's correspondence/retrieval on the synthetic data can be evaluated or parsed by an LLM and a consistent output format such as "Pass"/"Fail", similarity, or extraction can be enforced.

- Many such results can be aggregated and a metric can be derived which explains something like "% of passing evaluations", "average number of relevant details from the sources", "average cosine similarity", etc.

This idea of using LLMs to test out and quantify chatbot quality, known as [**"LLM-as-a-Judge,"**](https://arxiv.org/abs/2306.05685) allows for easy test specifications that align closely with human judgment and can be fine-tuned and replicated at scale.

**There are several popular frameworks for off-the-shelf judge formulations including:**
- [**RAGAs (short for RAG Assessment)**](https://docs.ragas.io/en/stable/), which offers a suite of great starting points for your own evaluation efforts.
- [**LangChain Evaluators**](https://python.langchain.com/docs/guides/evaluation/), which are similar first-party options with many implicitly-constructible agents.

Instead of using the chains as-is, we will instead expand on the ideas and evaluate our system with a more custom solution.

----

<br>

## **Part 3: [Assessment Prep]** Pairwise Evaluator

The following exercise will flesh out a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/docs/guides/evaluation/examples/comparisons). 

**To prepare for our RAG chain evaluation, we will need to:**

- Pull in our document index (the one we saved in the previous notebook).
- Recreate our RAG pipeline of choice.

**We will specifically be implementing a judge formulation with the following steps:**

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

**The chain should be a simple but powerful process that tests for the following objective:**

> ***Does my RAG chain outperform a narrow chatbot with limited document access.***





**This will be the system used for the final evaluation!** To see how this system is integrated into the autograder, please check out the implementation in [`frontend/server_app.py`](frontend/server_app.py).

<br>

### **Task 1:** Pull In Your Document Retreival Index

For this exercise, you will pull in the `docstore_index` file you created as part of your earlier notebook. The following cell should be able to load in the store as-is.

In [2]:
## Optional if pulling from the last notebook, the following should work
##   Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

embedder = NVIDIAEmbeddings(model="nvolveqa_40k")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"\nSample Chunk:\n\n{format_chunk(docs[len(docs)//2])}")

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
Constructed aggregate docstore with 225 chunks


Sample Chunk:

Paper: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

Summary: Large pre-trained language models have been shown to store factual knowledge
in their parameters, and achieve state-of-the-art results when fine-tuned on
downstream NLP tasks. However, their ability to access and precisely manipulate
knowledge is still limited, and hence on knowledge-intensive tasks, their
performance lags behind task-specific architectures. Additionally, providing
provenance for their decisions and updating their world knowledge remain open
research problems. Pre-trained models with a differentiable access mechanism to
explicit non-parametric memory can overcome this issue, but have so far been
only investigated for extractive downstream tasks. We explore a general-purpose
fine-tuning recipe for retrieval-augmented generation (RAG) -- models which
combine pre-trained parametric and non-parametric memory for language
generation. We introduce RAG models where the parametric memory is a
pre-trained seq2seq model and the non-parametric memory is a dense vector index
of Wikipedia, accessed with a pre-trained neural retriever. We compare two RAG
formulations, one which conditions on the same retrieved passages across the
whole generated sequence, the other can use different passages per token. We
fine-tune and evaluate our models on a wide range of knowledge-intensive NLP
tasks and set the state-of-the-art on three open domain QA tasks, outperforming
parametric seq2seq models and task-specific retrieve-and-extract architectures.
For language generation tasks, we find that RAG models generate more specific,
diverse and factual language than a state-of-the-art parametric-only seq2seq
baseline.

Page Body: RAG-S This 14th century work is divided into 3 sections: "Inferno", "Purgatorio" & "Paradiso"
For 2-way classiﬁcation, we compare against Thorne and Vlachos [57], who train RoBERTa [35]
to classify the claim as true or false given the gold evidence sentence. RAG achieves an accuracy
within 2.7% of this model, despite being supplied with only the claim and retrieving its own evidence.
We also analyze whether documents retrieved by RAG correspond to documents annotated as gold
evidence in FEVER. We calculate the overlap in article titles between the top k documents retrieved
by RAG and gold evidence annotations. We ﬁnd that the top retrieved document is from a gold article
in 71% of cases, and a gold article is present in the top 10 retrieved articles in 90% of cases.
4.5
Additional Results
Generation Diversity
Section 4.3 shows that RAG models are more factual and speciﬁc than
BART for Jeopardy question generation. Following recent work on diversity-promoting decoding

<br>

### **Task 2: [Exercise]** Pull In Your RAG Chain

Now that we have our index, we can recreate the RAG agent from the previous notebook! 

**Key Modifications:**
- To keep things simple, feel free to disregard the vectorstore-as-a-memory component. Incorporating it will require some more overhead and will make the exercise a bit more complicated.

In [64]:
# docs_chunks = format_chunk(docs[len(docs)//2])

In [93]:
docs[2].__dict__.values()

dict_values(["{'Published': '2019-05-24', 'Title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding', 'Authors': 'Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova', 'Summary': 'We introduce a new language representation model called BERT, which stands\\nfor Bidirectional Encoder Representations from Transformers. Unlike recent\\nlanguage representation models, BERT is designed to pre-train deep\\nbidirectional representations from unlabeled text by jointly conditioning on\\nboth left and right context in all layers. As a result, the pre-trained BERT\\nmodel can be fine-tuned with just one additional output layer to create\\nstate-of-the-art models for a wide range of tasks, such as question answering\\nand language inference, without substantial task-specific architecture\\nmodifications.\\n  BERT is conceptually simple and empirically powerful. It obtains new\\nstate-of-the-art results on eleven natural language processing tasks, including\

In [94]:
first_doc = docs[2].page_content

In [95]:
second_doc = docs[3].page_content

In [96]:
type(docs)
docs_chunks = []

In [97]:
docs_chunks = dict({'first_doc' : first_doc,'second_doc':second_doc})

In [98]:
docs_chunks

{'first_doc': "{'Published': '2019-05-24', 'Title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding', 'Authors': 'Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova', 'Summary': 'We introduce a new language representation model called BERT, which stands\\nfor Bidirectional Encoder Representations from Transformers. Unlike recent\\nlanguage representation models, BERT is designed to pre-train deep\\nbidirectional representations from unlabeled text by jointly conditioning on\\nboth left and right context in all layers. As a result, the pre-trained BERT\\nmodel can be fine-tuned with just one additional output layer to create\\nstate-of-the-art models for a wide range of tasks, such as question answering\\nand language inference, without substantial task-specific architecture\\nmodifications.\\n  BERT is conceptually simple and empirically powerful. It obtains new\\nstate-of-the-art results on eleven natural language processing tasks, including

In [19]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=100,
#     separators=["\n\n", "\n", ".", ";", ",", " ", ""],
# )
# ## Split the documents and also filter out stubs (overly short chunks)
# print("Chunking Documents")
# docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
# docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]


In [115]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################
## TODO: Update as necessary and add new components
llm = ChatNVIDIA(model='mixtral_8x7b') | StrOutputParser()
embedder = NVIDIAEmbeddings(model='nvolveqa_40k')
# vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]
#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
), ('user', '{input}')])

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#####################################################################
## TODO: Pull in your desired RAG Chain. Memory not necessary
context_getter = RunnableLambda(lambda x: x)
custom_chain =  RPrint("Hello World") | RunnableAssign({'context' : context_getter,'input': (lambda x:x)})

## Chain Specs: "Hello World" -> retrieval_chain 
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
  ## TODO
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter}) 

## Chain Specs: retrieval_chain -> generator_chain 
##   -> {"output" : <str>, ...} -> output_puller
generator_chain = custom_chain | {'output' : (lambda x:chat_prompt | llm  )} 
print(generator_chain)## TODO
generator_chain = generator_chain | output_puller  ## GIVEN

## END TODO
#####################################################################

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

first=RunnableLambda(...) middle=[RunnableAssign(mapper={
  context: RunnableLambda(lambda x: x),
  input: RunnableLambda(lambda x: x)
})] last={
  output: RunnableLambda(lambda x: chat_prompt | llm)
}
Hello World{'input': 'Tell me something interesting!', 'context': {'input': 'Tell me something interesting!'}}
Sure, I'd be happy to share something interesting with you! Did you know that documents can be retrieved and analyzed using natural language processing and machine learning techniques? This technology is called Document Retrieval, and it's used in various fields such as law, healthcare, and research to quickly and accurately find relevant information in large collections of documents.

For example, in the legal field, Document Retrieval can be used to search through thousands of legal documents to find cases that are relevant to a specific case being argued in court. This can save lawyers and judges a significant amount of time and resources.

In healthcare, Document Retrieval c

<br>

### **Step 3:** Generating Synthetic Question-Answer Pairs

In this section, we can implement the first few part of our evaluation routine:

- **Sample the RAG agent document pool to find two document chunks.**
- **Use those two document chunks to generate a synthetic "baseline" question-answer pair.**
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [45]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', '{input}')])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}", synth_questions[-1], "", sep='\n')
    pprint(synth_answers[-1], "", sep='\n')

QA Pair 1
Question: How do the Transformer model and the BERT model, both based on attention mechanisms, differ in their 
approach to language representation and processing?

Answer: The Transformer model, as described in the Paper: Attention Is All You Need, is a sequence transduction 
model that uses self-attention mechanisms to process input sequences, dispensing with recurrence and convolutions 
entirely. It modifies the self-attention sub-layer in the decoder stack to prevent positions from attending to 
subsequent positions, ensuring that the predictions for a position can only depend on the known outputs at previous
positions. The BERT model, on the other hand, as introduced in the Paper: BERT: Pre-training of Deep Bidirectional 
Transformers for Language Understanding, is a language representation model that pre-trains deep bidirectional 
representations from unlabeled text by jointly conditioning on both left and right context in all layers. BERT is 
designed to be fine-tuned with just one additional output layer for a wide range of tasks, such as question 
answering and language inference, without substantial task-specific architecture modifications. The Transformer 
model is used for sequence-to-sequence tasks like machine translation, while BERT is used for tasks that require 
understanding of language context, like question answering and language inference.

QA Pair 2
Question: How do BERT and Retrieval-Augmented Generation (RAG) models compare in their ability to handle 
knowledge-intensive NLP tasks?

Answer: BERT is a model that pre-trains deep bidirectional representations from unlabeled text by jointly 
conditioning on both left and right context in all layers. It can be fine-tuned with just one additional output 
layer to create state-of-the-art models for a wide range of tasks, including question answering and language 
inference, without substantial task-specific architecture modifications. However, its ability to access and 
precisely manipulate knowledge is still limited, and providing provenance for its decisions and updating its world 
knowledge remain open research problems. On the other hand, RAG models combine pre-trained parametric and 
non-parametric memory for language generation. The non-parametric memory in RAG models is a dense vector index of 
Wikipedia, accessed with a pre-trained neural retriever. RAG models have been shown to outperform parametric 
seq2seq models and task-specific retrieve-and-extract architectures on a wide range of knowledge-intensive NLP 
tasks. For language generation tasks, RAG models generate more specific, diverse, and factual language than a 
state-of-the-art parametric-only seq2seq baseline. RAG models can also go beyond simple extractive QA and answer 
questions with free-form, abstractive text generation.

QA Pair 3
Question: How does the Mistral 7B model compare to Llama 2 and Llama 1 in various tasks, and what techniques does 
it use for faster inference and handling sequences of arbitrary length?

Answer: The Mistral 7B model outperforms both Llama 2 13B and Llama 1 34B in various benchmarks, particularly in 
reasoning, mathematics, and code generation tasks. It achieves this by utilizing grouped-query attention (GQA) for 
faster inference and sliding window attention (SWA) to effectively handle sequences of arbitrary length with a 
reduced inference cost. The model also provides a fine-tuned version, Mistral 7B -- Instruct, which surpasses the 
Llama 2 13B -- Chat model on human and automated benchmarks. Additionally, Mistral 7B has shown a preference of 
5020 to 4143 over Llama 2 13B in a comparison of outputs from both models. However, Mistral 7B performs similarly 
to Llama 2 13B on knowledge benchmarks, likely due to its limited parameter count.

In [110]:
# usr_msg

<br>

### **Step 4: [Exercise]** Answer The Synthetic Questions

In this section, we can implement the third part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use the RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [111]:
synth_questions

['Question: How do the Transformer model and the BERT model, both based on attention mechanisms, differ in their approach to language representation and processing?',
 'Question: How do BERT and Retrieval-Augmented Generation (RAG) models compare in their ability to handle knowledge-intensive NLP tasks?',
 'Question: How does the Mistral 7B model compare to Llama 2 and Llama 1 in various tasks, and what techniques does it use for faster inference and handling sequences of arbitrary length?']

In [118]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    rag_answer = ""
    rag_answer = (rag_chain).invoke({'input': q})
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')
    

Hello World{'input': {'input': 'Question: How do the Transformer model and the BERT model, both based on attention mechanisms, differ in their approach to language representation and processing?'}, 'context': {'input': {'input': 'Question: How do the Transformer model and the BERT model, both based on attention mechanisms, differ in their approach to language representation and processing?'}}}


QA Pair 1
Question: How do the Transformer model and the BERT model, both based on attention mechanisms, differ in their 
approach to language representation and processing?

RAG Answer: The Transformer model and the BERT model are both built on attention mechanisms, but they differ in 
their approach to language representation and processing.

The Transformer model, introduced in the paper "Attention is All You Need" by Vaswani et al., 2017, is a model 
architecture for neural machine translation. It uses self-attention mechanisms to process input sequences, allowing
for more parallelization and reducing the need for recurrence. The Transformer model represents input text as 
continuous vectors and processes them using self-attention layers, enabling it to capture long-range dependencies 
in the data.

On the other hand, BERT (Bidirectional Encoder Representations from Transformers), presented in the paper "BERT: 
Pre-training of Deep Bidirectional Transformers for Language Understanding" by Devlin et al., 2018, is a 
pre-trained language model that uses bidirectional transformers. BERT's primary innovation is the introduction of 
bidirectional self-attention, which processes input text in both directions simultaneously, capturing context from 
both left and right sides of a word. This approach allows BERT to generate more context-aware language 
representations compared to the Transformer model.

In summary, while both models are based on attention mechanisms, the Transformer model focuses on parallel 
processing and capturing long-range dependencies, while BERT emphasizes context-aware language representations 
through bidirectional processing.

Sources:

1. Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). 
Attention is all you need. In Advances in neural information processing systems (pp. 5998-6008).
2. Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2018). BERT: Pre-training of deep bidirectional transformers
for language understanding. In Proceedings of the 2019 Conference of the North American Chapter of the Association 
for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers) (pp. 4171-4186).

Hello World{'input': {'input': 'Question: How do BERT and Retrieval-Augmented Generation (RAG) models compare in their ability to handle knowledge-intensive NLP tasks?'}, 'context': {'input': {'input': 'Question: How do BERT and Retrieval-Augmented Generation (RAG) models compare in their ability to handle knowledge-intensive NLP tasks?'}}}


QA Pair 2
Question: How do BERT and Retrieval-Augmented Generation (RAG) models compare in their ability to handle 
knowledge-intensive NLP tasks?

RAG Answer: Based on the information I retrieved, BERT and Retrieval-Augmented Generation (RAG) models are two 
different approaches to handling knowledge-intensive NLP tasks.

BERT, which stands for Bidirectional Encoder Representations from Transformers, is a popular transformer-based 
machine learning technique for NLP tasks. It pre-trains deep bidirectional representations from unlabeled text by 
jointly conditioning on both left and right context in all layers. This allows BERT to create more meaningful 
representations of words, taking into account their context within a sentence. BERT has been successful in a 
variety of NLP tasks, such as question answering, sentiment analysis, and named entity recognition.

On the other hand, Retrieval-Augmented Generation (RAG) models are a more recent approach that combines dense 
retrieval with sequence generation. RAG models first retrieve relevant documents from a large corpus, and then 
generate an answer based on the retrieved documents and the input query. This allows RAG models to handle 
knowledge-intensive NLP tasks that require access to a large external knowledge base.

In terms of their ability to handle knowledge-intensive NLP tasks, RAG models have an advantage over BERT because 
they can access a larger knowledge base beyond the pre-training corpus. However, RAG models require an additional 
step of document retrieval, which can add to the computational cost and latency of the model.

So, to summarize, BERT and RAG models are two different approaches to handling knowledge-intensive NLP tasks, with 
BERT relying solely on pre-training and RAG models accessing a larger knowledge base through document retrieval.

Sources used:

* Devlin, J., Chang, M.W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers 
for Language Understanding. arXiv preprint arXiv:1810.04805.
* Lewis, P., Liu, Y., Goyal, N., & Acharya, U. (2020). Retrieval-Augmented Generation for Knowledge-Intensive NLP 
Tasks. arXiv preprint arXiv:2005.11401.

Hello World{'input': {'input': 'Question: How does the Mistral 7B model compare to Llama 2 and Llama 1 in various tasks, and what techniques does it use for faster inference and handling sequences of arbitrary length?'}, 'context': {'input': {'input': 'Question: How does the Mistral 7B model compare to Llama 2 and Llama 1 in various tasks, and what techniques does it use for faster inference and handling sequences of arbitrary length?'}}}


QA Pair 3
Question: How does the Mistral 7B model compare to Llama 2 and Llama 1 in various tasks, and what techniques does 
it use for faster inference and handling sequences of arbitrary length?

RAG Answer: I'm here to help answer your questions to the best of my ability using the information provided. 
However, I don't have access to any documents or databases outside of what has been specifically provided to me in 
the context of this conversation.

Based on the information you've given me, I can tell you that there are no specific details about the Mistral 7B 
model, Llama 2, or Llama 1 provided, so I'm unable to compare their performance in various tasks.

As for the techniques Mistral 7B uses for faster inference and handling sequences of arbitrary length, the 
information is also not provided. Generally, models use techniques like dynamic programming, attention mechanisms, 
or caching to handle sequences of arbitrary length more efficiently. For faster inference, models might use methods
like weight pruning, quantization, or knowledge distillation. But without specific information about the Mistral 7B
model, I can't say for sure what techniques it uses.

I hope this helps, and please let me know if you have any other questions!

<br>

### **Step 5:** Implement A Human Preference Metric

In this section, we can implement the fourth part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- **Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."**

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [ ]:
eval_instruction = """
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[0] The second answer lies, does not answer the question, or is inferior to the first answer.
[1] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification
"""

eval_prompt = ChatPromptTemplate.from_messages([
    ('system', eval_instruction), ('user', '{input}')
])

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    usr_msg = f"Question: {q}\n\nAnswer 1: {a_synth}\n\n Answer 2: {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'input': usr_msg})]
    # pprint(f"Synth Answer: {a_synth}\n\n")
    # pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

<br>

**Congratulations! We now have an LLM system that reasons about our pipeline and tries to evaluate it!** Now that we have some judge results, we can simply aggregate the results and see how often our formulation was according to an LLM:

In [ ]:
pref_score = sum(("[1]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

----

<br>

## **Part 4:** Advanced Formulations

The exercise above was meant to prepare you for the final assessment of the course and showcased a simple but effective evaluator chain. The objective and implementation details were provided for you, and the logic for using it probably makes sense now that you've seen it in action. 

With that being said, this metric was merely a product of us specifying:
- **What kind of behavior is important for our pipeline to have?**
- **What do we need to do in order to exhibit and evaluate this behavior?**

From these two questions, we could have come up with plenty of other evaluation metrics that could have assessed different attributes, incorporated different evaluator chain techniques, and even required different pipeline organization strategies. Though far from an exhaustive list, some common formulations you will likely come across may include:

- **Style Evaluation:** Some evaluation formulations can be as simple as "let me ask some questions and see if the output feels desirable." This might be used to see whether a chatbot "acts like it's supposed to" based on a description provided to a judge LLM. We're using quotations since this kind of assessment can reasonably be achieved with nothing but prompt engineering and a while loop.

- **Ground-Truth Evaluation:** In our chain, we used synthetic generation to create some random questions and answers using a sampling strategy, but in reality you may actually have some representative questions and answers that you need your chatbot to consistently get right! In this case, a modification of the exercise chain above should be implemented and closely monitored as you develop your pipelines.

- **Retrieval/Augmentation Evaluation:** This course made many assumptions about what kinds of preprocessing and prompting steps would be good for your pipelines, and much of this was determined by experimentation. Factors such as document preprocessing, chunking strategies, model selection, and prompt specification all played important roles, so creating metrics to validate these decisions may be of interest. This kind of metric might require your pipeline to output your context chunks or may even rely solely on embedding similarity comparisons, so keep this in mind when trying to implement a chain that works with multiple evaluation strategies. Consider the [**RagasEvaluatorChain**](https://docs.ragas.io/en/latest/howtos/integrations/langchain.html) abstraction as a decent starting point for making an custom generalizable evaluation routine. 

- **Trajectory Evaluation:** Using more advanced agent formulations, you can implement multiple-query strategies that assume the presence of conversational memory. With this, you can implement an evaluation agent which can:
    - Ask a series of questions in order to evaluate how well the agent is able to adapt and cater to the scenario. This kind of system generally considers a series of correspondence and aims to tease out and evaluate a "trajectory" of how the agent navigated the conversation. The [**LangChain Trajectory Evaluators documentation**](https://python.langchain.com/docs/guides/evaluation/trajectory/) is a good starting point.
    - Alternatively, you could also implement an evaluation agent that tries to achieve objectives by interacting with the chatbot. Such an agent can output whether they were able to navigate to their solution in a natural manner, and can even be used to generate a report about the percieved performance. The [**LangChain Agents documentation**](https://python.langchain.com/docs/modules/agents/concepts) is a good starting point!

<br>

At the end of the day, just make sure to use the tools you have at your disposal appropriately. By this point in the course, you should already be well-acquainted with the LLM core value propositions: **They're powerful, scalable, predictable, controllable, and orchestratable... but will act unpredictably when you just expect them to work by default.** Assess your needs, formulate and validate your pipelines, give enough information, and add as much control as you can to make your system work consistently, efficiently, and effectively.

----

<br>

## **Part 5: [Assessment]** Evaluating For Credit

Welcome to the last exercise of the course! Hopefully you've enjoyed the material and are ready to actually get credit for these notebooks! For this part:

- **Make sure you're in the course environment**
- **Make sure `docstore_index/` has been uploaded to the course environment...**
    - **...and contains [at least one Arxiv paper](https://arxiv.org/search/advanced) which has been updated less than 30 days ago.**
- **Make sure [`35_langserve.ipynb`]() is not occupying port 9012 with a running FastAPI service**

**Objective:** On launch, [**`frontend/server_app.py`**](frontend/server_app.py) had several lines of code which trigger the course pass condition. Once the service was deemed healthy, that sequence of code was replaced with `## secret` by another microservice. Your objective is to invoke that series of commands by using your pipeline to pass the Evaluation check! Recall [`35_langserve.ipynb`](35_langserve.ipynb) and use it as a starting example! As a recommendation, consider duplicating it so that you can keep the original as an authoritative reference. 

**Once Finished:** While your course environment is still open, please navigate back to your course environment launcher area and click the **"Assess Task"** button! After that, you're all done!

----

<br>

## <font color="#76b900">**Congratulations On Completing The Course**</font>

Hopefully this course was not only exciting and challenging, but also adequately prepared you for work on the cutting edge of LLM and RAG system development! Going forward, you should have the skills necessary to tackle industry-level challenges and explore RAG deployment with open-source models and frameworks.

**Some NVIDIA-specific releases related to this that you may find interesting include:**
- [**NVIDIA NeMo Microservices**](https://developer.nvidia.com/nemo-microservices-early-access), which offers microservice spinup routines that can be deployed on local compute and function similar to AI Playground.
- [**NVIDIA AI Workbench**](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/workbench/), which allows for quick and simple model customization workflows that can greatly improve RAG model components for your specific use-cases.
- [**TensorRT-LLM**](https://github.com/NVIDIA/TensorRT-LLM) is the current recommended framework for deploying GPU-accelerated LLM model engines in production settings.
- [**NVIDIA's Generative AI Examples Repo**](https://github.com/NVIDIA/GenerativeAIExamples), which includes the current canonical microservice example application and will be updated with new resources as new production workflows get released.
- [**The Knowledge-Based Chatbot Technical Brief**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief) which discusses additional publicly-accessible details on productionalizing RAG systems.

**Additionally, some key topics you may be interested in delving more into include:**
- [**LlamaIndex**](https://www.llamaindex.ai/), which has strong components that can augment and occasionally improve upon the LangChain RAG features.
- [**LangSmith**](https://docs.smith.langchain.com/), an upcoming agent productionalization service offered by LangChain.
- [**Gradio**](https://www.gradio.app/), though touched on in the course, has many more interface options which will be worth investigating. For inspiration, consider checking out [**HuggingFace Spaces**](https://huggingface.co/spaces) for examples.

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>